# Group THE BOYS 1D ML project
1. Loy Pek Yong 1004475
2. Lim Yongqing 1005955


# Question 1

## Importing libraries + reading data + preprocessing

In [49]:
# import libraries
import pandas as pd
import numpy as np
print(np.__version__)
print(pd.__version__)
# read data
# ES_train = pd.read_csv("./ES/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
# ES_dev_in = pd.read_csv("./ES/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
# ES_dev_out = pd.read_csv("./ES/dev.out", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)

# RU_train = pd.read_csv("./RU/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
# RU_dev_in = pd.read_csv("./RU/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
# RU_dev_out = pd.read_csv("./RU/dev.out", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
with open('./ES/train', 'r', encoding = 'utf8') as f:
    r1 = [lines.rstrip("\n") for lines in f.readlines()]
ES_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r1]

with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_in_tup = [ lines if lines != "" else "null" for lines in r2]

with open('./ES/dev.out', 'r', encoding = 'utf8') as f:
    r3 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r3]



with open('./RU/train', 'r', encoding = 'utf8') as f:
    r4 = [lines.rstrip("\n") for lines in f.readlines()]
RU_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r4]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    r5 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_in_tup = [ lines if lines != "" else "null" for lines in r5]

with open('./RU/dev.out', 'r', encoding = 'utf8') as f:
    r6 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r6]


1.25.2
2.0.3


## Changing to dataframe

In [50]:
ES_train = pd.DataFrame(data = ES_train_tup, columns = ["word", "label"])
ES_dev_in = pd.DataFrame(data = ES_dev_in_tup, columns = ["word"])
ES_dev_out = pd.DataFrame(data = ES_dev_out_tup, columns = ["word", "label"])

RU_train = pd.DataFrame(data = RU_train_tup, columns = ["word", "label"])
RU_dev_in = pd.DataFrame(data = RU_dev_in_tup, columns = ["word"])
RU_dev_out = pd.DataFrame(data = RU_dev_out_tup, columns = ["word", "label"])

# Part 1 of q1

## function that returns the emission parameters with input of training set

In [51]:
def train_estimate_emission_parameters(train):
    # word_column = train["word"]
    # label_column = train["label"]

    label_count = {}
    emission_count = {}

    for row in range(train.shape[0]):
        word = train.iloc[row, 0]
        label = train.iloc[row, 1]
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1

        if (word, label) not in emission_count:
            emission_count[(word, label)] = 1
        else:
            emission_count[(word, label)] += 1

    emission_parameter = {}
    for word, label in emission_count:
        emission_parameter[(word, label)] = emission_count[(word, label)] / label_count[label]

    return emission_parameter, emission_count, label_count

# debug
# emission_parameter, emission_count, label_count = train_estimate_emission_parameters(ES_train)
# for i in emission_parameter:
#     print(i, emission_parameter[i])

In [52]:
train_estimate_emission_parameters(ES_train)

({('Estuvimos', 'O'): 0.0002066471499913897,
  ('hace', 'O'): 0.0008954709832960221,
  ('poco', 'O'): 0.0018942655415877389,
  ('mi', 'O'): 0.0024797657998966763,
  ('pareja', 'O'): 0.00044773549164801103,
  ('y', 'O'): 0.035267780265197175,
  ('yo', 'O'): 0.0012398828999483382,
  ('comiendo', 'O'): 0.00034441191665231615,
  ('resultó', 'O'): 0.00013776476666092647,
  ('todo', 'O'): 0.003960737041501636,
  ('muy', 'O'): 0.01363871189943172,
  ('bien', 'O'): 0.005682796624763217,
  (',', 'O'): 0.05731014293094541,
  ('tanto', 'O'): 0.001343206474944033,
  ('la', 'O'): 0.02600309970724987,
  ('comida', 'B-positive'): 0.1456896551724138,
  ('el', 'O'): 0.022111245049078698,
  ('vino', 'B-positive'): 0.005172413793103448,
  ('trato', 'B-positive'): 0.03793103448275862,
  ('decoración', 'B-positive'): 0.00603448275862069,
  ('…', 'O'): 0.0015498536249354228,
  ('nos', 'O'): 0.005028413983123816,
  ('gustó', 'O'): 0.00037885310831754776,
  ('mucho', 'O'): 0.0018253831582572756,
  ('.', 'O'):

# Part 2 of q1

## function that returns the emission parameters with input set

In [53]:
def upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_counts, label_counts):
    # predict the label of the test set
    k = 1
    special_word = '#UNK#'
    test_copy = test_set.copy()
    #train word list is all the first argument in keys of emission counts
    train_word_list= [word for word, label in emission_counts.keys()]
    for i in range(len(test_copy["word"])):
        word = test_copy["word"][i]
        if word not in train_word_list:
            test_copy["word"][i] = special_word
#             test_copy["label"][i] = "O"
#             print(word, test_copy["label"][i])
            # k += 1
    for word in test_copy["word"]:
        for label in label_counts:
            if word == special_word:
                emission_parameter[(word, label)] = k / (label_counts[label] + k)
            elif ( (word, label) in emission_parameter ):
                emission_parameter[(word, label)] = emission_counts[(word, label)] / (label_counts[label] + k)
                
    print(emission_parameter)        
    return emission_parameter

# Part 3 of q1

In [54]:
def sentiment_analysis(training_set, test_set):
    emission_parameter, emission_count, label_count = train_estimate_emission_parameters(training_set)
    emission_parameter = upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_count, label_count)
    test_set["label"] = ""

    #label equals to argmax of emission parameter
    for row in range(len(test_set)):
        word = test_set.iloc[row]["word"]
        max_val = 0
        max_label = ""
        train_word_list = [word for word, label in emission_count.keys()]
        for label in label_count:
            if (word, label) in emission_parameter:
                if emission_parameter[(word, label)] > max_val:
                    max_val = emission_parameter[(word, label)]
                    max_label = label
                    
            else:
                if word not in train_word_list:
                    # get max label from word #UNK#
                    if ("#UNK#", label) in emission_parameter:
                        if emission_parameter[("#UNK#", label)] > max_val:
                            max_val = emission_parameter[("#UNK#", label)]
                            max_label = label
        test_set.loc[row, "label"] = max_label
                    
    return test_set


def precision(predicted, test_set):
    # calculate the precision
    correct = 0
    total = 0
    for row in range(len(test_set)):
        if test_set.iloc[row]["label"] == predicted.iloc[row]["label"]:
            correct += 1
        total += 1
    return correct/total if total > 0 else 0

def recall(predicted, compare_set):
    # calculate the recall
    correct = 0
    total = 0
    for row in range(len(compare_set)):
        if compare_set.iloc[row]["label"] == predicted.iloc[row]["label"]:
            correct += 1
        total += 1
    return correct/total if total > 0 else 0

def f1_score(precision, recall):
    # calculate the f1 score
    return 2*precision*recall/(precision+recall)

## Predicting + saving to csv

In [55]:
ES_dev_p1_out = sentiment_analysis(ES_train, ES_dev_in)
ES_dev_p1_out_r = ES_dev_p1_out.replace('null', pd.NA)
ES_dev_p1_out_r.to_csv("./ES/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
# with open("./ES/dev.p1.out", mode='w', encoding='utf-8') as ES_dev_p1_out:
#     ES_dev_p1_out.write("\n")
# ES_dev_p1_out.close()
# ES_dev_p1_out.to_csv("./ES/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
    
RU_dev_p1_out = sentiment_analysis(RU_train, RU_dev_in)
RU_dev_p1_out_r = RU_dev_p1_out.replace('null', pd.NA)
RU_dev_p1_out_r.to_csv("./RU/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)


{('Estuvimos', 'O'): 0.0002066471499913897, ('hace', 'O'): 0.0008954401432704229, ('poco', 'O'): 0.0018942003030720485, ('mi', 'O'): 0.0024796803967488635, ('pareja', 'O'): 0.00044772007163521146, ('y', 'O'): 0.0352665656426505, ('yo', 'O'): 0.0012398401983744318, ('comiendo', 'O'): 0.00034440005510400884, ('resultó', 'O'): 0.00013776476666092647, ('todo', 'O'): 0.0039606006336961016, ('muy', 'O'): 0.013638242182118749, ('bien', 'O'): 0.005682600909216145, (',', 'O'): 0.05730816916930707, ('tanto', 'O'): 0.0013431602149056344, ('la', 'O'): 0.026002204160352666, ('comida', 'B-positive'): 0.14556416881998277, ('el', 'O'): 0.022110483537677365, ('vino', 'B-positive'): 0.00516795865633075, ('trato', 'B-positive'): 0.03789836347975883, ('decoración', 'B-positive'): 0.006029285099052541, ('…', 'O'): 0.0015498002479680396, ('nos', 'O'): 0.005028240804518529, ('gustó', 'O'): 0.0003788400606144097, ('mucho', 'O'): 0.0018253202920512468, ('.', 'O'): 0.055896128943380634, ('null', 'null'): 0.9994

In [56]:
ES_dev_p1_out.iloc[:27, :]

,word,label
0,Plato,B-negative
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
5,abundante,O
6,de,I-positive
7,más,O
8,",",O
9,pero,I-neutral


# Scoring

In [57]:
print("ES:")
p = precision(ES_dev_p1_out, ES_dev_out)
r = recall(ES_dev_p1_out, ES_dev_out)
f1 = f1_score(p, r)
print("precision is:", p)
print("recall is:",  r)
print("f1 score is:", f1)

print()

print("RU:")
p = precision(RU_dev_p1_out, RU_dev_out)
r = recall(RU_dev_p1_out, RU_dev_out)
f1 = f1_score(p, r)
print("precision is:", p)
print("recall is:",  r)
print("f1 score is:", f1)


ES:
precision is: 0.6522498907820009
recall is: 0.6522498907820009
f1 score is: 0.6522498907820009

RU:
precision is: 0.7105038428693424
recall is: 0.7105038428693424
f1 score is: 0.7105038428693423


In [58]:
ES_dev_p1_out

,word,label
0,Plato,B-negative
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4573,dejarte,I-neutral
4574,la,I-positive
4575,cartera,I-neutral
4576,.,O


In [59]:
ES_dev_out

,word,label
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4573,dejarte,O
4574,la,O
4575,cartera,O
4576,.,O


# 

# Question 2

## Reading data + preprocessing

In [60]:
with open('./ES/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
EStrain = [lines.rstrip("\n") for lines in file]
EStrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in EStrain ]

with open('./RU/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUtrain = [lines.rstrip("\n") for lines in file]
RUtrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in RUtrain ]

# Part 1 of q2

## Creating reference transition dictionary (all transitions)

In [61]:
transit_dict = {
    ("O", "O"): 0,
    ("O", "B-positive"):0,
    ("O", "B-negative"):0,
    ("O", "B-neutral"):0,
    ("O", "I-positive"):0,
    ("O", "I-negative"):0,
    ("O", "I-neutral"):0,
    ("O", "START"):0,
    ("O", "STOP"):0,
    
    ("B-positive", "O"):0,
    ("B-positive", "B-positive"):0,
    ("B-positive", "B-negative"):0,
    ("B-positive", "B-neutral"):0,
    ("B-positive", "I-positive"):0,
    ("B-positive", "I-negative"):0,
    ("B-positive", "I-neutral"):0,
    ("B-positive", "START"):0,
    ("B-positive", "STOP"):0,
    
    
    ("B-negative", "O"):0,
    ("B-negative", "B-positive"):0,
    ("B-negative", "B-negative"):0,
    ("B-negative", "B-neutral"):0,
    ("B-negative", "I-positive"):0,
    ("B-negative", "I-negative"):0,
    ("B-negative", "I-neutral"):0,
    ("B-negative", "START"):0,
    ("B-negative", "STOP"):0,
    
    ("B-neutral", "O"):0,
    ("B-neutral", "B-positive"):0,
    ("B-neutral", "B-negative"):0,
    ("B-neutral", "B-neutral"):0,
    ("B-neutral", "I-positive"):0,
    ("B-neutral", "I-negative"):0,
    ("B-neutral", "I-neutral"):0,
    ("B-neutral", "START"):0,
    ("B-neutral", "STOP"):0,
    
    ("I-positive", "O"):0,
    ("I-positive", "B-positive"):0,
    ("I-positive", "B-negative"):0,
    ("I-positive", "B-neutral"):0,
    ("I-positive", "I-positive"):0,
    ("I-positive", "I-negative"):0,
    ("I-positive", "I-neutral"):0,
    ("I-positive", "START"):0,
    ("I-positive", "STOP"):0,
    
    ("I-negative", "O"):0,
    ("I-negative", "B-positive"):0,
    ("I-negative", "B-negative"):0,
    ("I-negative", "B-neutral"):0,
    ("I-negative", "I-positive"):0,
    ("I-negative", "I-negative"):0,
    ("I-negative", "I-neutral"):0,
    ("I-negative", "START"):0,
    ("I-negative", "STOP"):0,
    
    ("I-neutral", "O"):0,
    ("I-neutral", "B-positive"):0,
    ("I-neutral", "B-negative"):0,
    ("I-neutral", "B-neutral"):0,
    ("I-neutral", "I-positive"):0,
    ("I-neutral", "I-negative"):0,
    ("I-neutral", "I-neutral"):0,
    ("I-neutral", "START"):0,
    ("I-neutral", "STOP"):0,
    
    ("START", "O"):0,
    ("START", "B-positive"):0,
    ("START", "B-negative"):0,
    ("START", "B-neutral"):0,
    ("START", "I-positive"):0,
    ("START", "I-negative"):0,
    ("START", "I-neutral"):0,
    ("START", "START"):0,
    ("START", "STOP"):0,
    
    ("STOP", "O"):0,
    ("STOP", "B-positive"):0,
    ("STOP", "B-negative"):0,
    ("STOP", "B-neutral"):0,
    ("STOP", "I-positive"):0,
    ("STOP", "I-negative"):0,
    ("STOP", "I-neutral"):0,
    ("STOP", "START"):0,
    ("STOP", "STOP"):0,
               
}

## Function to calculate number of u (init state) to v (next state) transitions

In [62]:
"""
Assumptions made: 

1. Everything is in tuple pairs except the word "STOP" which signifies the end of a review
2. Transition probabilities from "STOP" to any other states are 0 but still included in the dictionary as required by question



Variables:
1. u --> initial state (e.g "O", "B-neutral", "I-positive" etc.)
2. v --> next state (e.g "O", "B-neutral", "I-positive" etc.)
3. hasbreak --> signifies the end of a review (boolean)

"""

def calculate_transition(data, transit_dict):
    updated_dict = transit_dict.copy()
    hasbreak = True
    for word in data:
        if ( hasbreak == True ):
            u = "START"
            
        if ( word != "STOP" and (hasbreak == True) ):
            v = word[1]
            updated_dict[(u, v)] += 1
            hasbreak = False
            
        elif ( word != "STOP" and (hasbreak == False) ):
            u = v
            v = word[1]
            updated_dict[(u, v)] += 1
        
        elif ( word == "STOP" ):
            u = v
            v = "STOP"
            updated_dict[(u, v)] += 1
            hasbreak = True
            
    return updated_dict

## Update for ES and RU

In [63]:
ES_updated_tdict = calculate_transition(EStrain_tup, transit_dict)
RU_updated_tdict = calculate_transition(RUtrain_tup, transit_dict)

## Function to convert above calculation to probability

In [64]:
def transition_param(updated_tdict):
    all_states = ['O', 'B-positive', 'B-negative', 'B-neutral', 'I-positive', 'I-negative', 'I-neutral', 'START', 'STOP']
    q_dict = {}
    temp_sum = 0
    for state in all_states:
        total = sum(value for key, value in updated_tdict.items() if key[1] == state)
        for key, value in updated_tdict.items():
            if ( (key[1] == state) and ( total != 0) ):
                q_dict[f"{state}|{key[0]}"] = value/total
            elif ( (key[1] == state) and ( total == 0) ):
                q_dict[f"{state}|{key[0]}"] = 0
    return q_dict

## *Below is the final parameters to be used*

*Format is v|u (next state given current state)*

In [65]:
ES_transit = transition_param(ES_updated_tdict)
RU_transit = transition_param(RU_updated_tdict)